# Finding our best performing salespeople and products

## Introduction

**Business Context.** You work for AdventureWorks, a company that sells outdoor sporting equipment. The company has many different locations and has been recording the sales of different locations on various products. You, their new data scientist, have been tasked with the question: **"What are our best products and sales people and how can use this to improve our other products and sales people with this information?"**

You have been given access to their database with documentation from the IT department. Your job is to extract meaningful insights from this database to help increase sales. First, you will look at the best products and try to see how different products behave in different categories. Second, you will analyze the best salespeople to see if commission percentage motivates them to sell more.

**Business Problem.** Your task is to query the companies database and perform simple satistical analys directly using ```SQL```.

**Analytical Context.** You are given a database (stored in the ```data``` folder) containing all the databases for the company. To simulate this in a real world setting, we will first create the database in the cloud by:

1. Creating an RDS instance on AWS to store a PostgreSQL database
2. Uploading the database using the ```install.sql``` file provided

The company has been pretty vague about how they expect you to extract insights, but you have come up with the following plan of attack:

1. Look at how product ratings and total sales are related
2. See how products sell in different subcategories (bikes, helmets, socks, etc.)
3. Calculating which salespeople have performed the best in the past year
4. Seeing if total sales are correlated with commission percentage

Of course, this is only your initial plan of attack. As you explore the database, your strategy will change.

## Setting up AWS

For this case, we will be simulating as if the company has its data in the cloud and will be doing everything through the terminal using `psql`.

### Question:

Repeat the steps in case 12.3 to create a new RDS instance with a Postgres database.

## Overview of the data

The data for the course is contained in the ```./data``` directory, but it is the ```AdventureWorks``` sample database provided by Microsoft. We will be focusing on the ```Sales``` and ```Production``` schemas. A brief overview of our tables of interest is presented here, but a complete documentation can be found [here](https://dataedo.com/download/AdventureWorks.pdf). The tables you should pay particular attention to (and look at in the documentation) are:

- **Production.Product**
- **Production.ProductReview**
- **Production.ProductModelProductDescriptionCulture**
- **Production.ProductDescription**
- **Sales.SalesPerson**
- **Sales.SalesOrderHeader**
- **Sales.SalesOrderDetail**

Regardless, when doing the exercises, **you should constantly refer to the documentation above**. You should also focus on building your queries one step at a time. If you have 3 ```JOIN```s to make in a query, you should start with the first ```SELECT``` and perform the ```JOIN```s one at a time and output them to make sure they are doing the correct thing.

### Question:

Upload the database to your server using the following two commands. The first will create the database and the second will upload the database. **Note:** You will have to be in the folder containing the case. After it is uploaded explore the database by performing some basic queries to get a better sense of the database. At the end, write a query to extract the first 5 rows of the table containing the names of the product descriptions. 


Note: You should be able to replace the "+" with "|" in your ```psql``` output to format it nicely as a Markdown table.

```bash
psql -c "CREATE DATABASE \"Adventureworks\";"
psql -d Adventureworks < install.sql
```

**Answer.** One possible solution is given below:

```mysql
SELECT * FROM .ProductDescription LIMIT 5;
```

productdescriptionid |                                                                       description                                                                       |               rowguid                |    modifieddate     
----------------------|---------------------------------------------------------------------------------------------------------------------------------------------------------|--------------------------------------|---------------------
                    3 | Chromoly steel.                                                                                                                                         | 301eed3a-1a82-4855-99cb-2afe8290d641 | 2013-04-30 00:00:00
                    4 | Aluminum alloy cups; large diameter spindle.                                                                                                            | dfeba528-da11-4650-9d86-cafda7294eb0 | 2013-04-30 00:00:00
                    5 | Aluminum alloy cups and a hollow axle.                                                                                                                  | f7178da7-1a7e-4997-8470-06737181305e | 2013-04-30 00:00:00
                    8 | Suitable for any type of riding, on or off-road. Fits any budget. Smooth-shifting with a comfortable ride.                                              | 8e6746e5-ad97-46e2-bd24-fcea075c3b52 | 2013-04-30 00:00:00
                   64 | This bike delivers a high-level of performance on a budget. It is responsive and maneuverable, and offers peace-of-mind when you decide to go off-road. | 7b1c4e90-85e2-4792-b47b-e0c424e2ec94 | 2013-04-30 00:00:00

## My conventions

_I decided to use ***psycopg2*** to connect to the database and eventually use pandas to check consistency, which will require an ad-hoc function called ```runMyquery```_

In [1]:
import psycopg2
import pandas as pd
#Limit 50 rows
pd.options.display.max_columns=50


In [2]:

my_connection=psycopg2.connect(user='postgres',
                              password='jV4QsGwfbz3rSphTSA9W',#password
                              host='ds4a-ggoni.cgqtq1pl7udv.us-east-2.rds.amazonaws.com', #my RDS instance
                              port='5432',
                              database='adventureworks')#database

In [3]:
def runMyQuery(my_query):
    df=pd.read_sql(my_query,my_connection)
    return df

## Finding our best sellers

The company is curious which of their products is the most popular among the customers. You first idea is to see if the average rating given in reviews is correlated with the number of sales in that product. This will help guide future decision making.

In [4]:
Ex_a="SELECT * FROM Production.Product LIMIT 5"

In [5]:
runMyQuery(Ex_a)

,productid,name,productnumber,makeflag,finishedgoodsflag,color,safetystocklevel,reorderpoint,standardcost,listprice,size,sizeunitmeasurecode,weightunitmeasurecode,weight,daystomanufacture,productline,class,style,productsubcategoryid,productmodelid,sellstartdate,sellenddate,discontinueddate,rowguid,modifieddate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,0.0,None,None,None,None,0,None,None,None,None,None,2008-04-30,None,None,694215b7-08f7-4c0d-acb1-d734ba44c0c8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,None,0,None,None,None,None,None,2008-04-30,None,None,58ae3c20-4f3a-4749-a7d4-d568806cc537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,None,1,None,None,None,None,None,2008-04-30,None,None,9c21aed2-5bfa-4f18-bcb8-f11638dc2e4e,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,None,0,None,None,None,None,None,2008-04-30,None,None,ecfed6cb-51ff-49b5-b06c-7d8ac834db8b,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,None,1,None,None,None,None,None,2008-04-30,None,None,e73e9750-603b-4131-89f5-3dd15ed5ff80,2014-02-08 10:01:36.827


### Exercise 1

Using the ```Production.Product``` and ```Production.ProductReview``` tables, ```JOIN``` them and rank the products according to their average review ratings. What are the names and IDs of the top products? Limit results to 5 rows and order by decreasing rating.

**Answer.**

In [6]:
queryEx1="SELECT DISTINCT PP.productid,name,ROUND(AVG(rating),1) AS meanRating\
        FROM Production.ProductReview as PR\
        INNER JOIN Production.Product as PP\
        ON PP.productid=PR.productid\
        GROUP BY PP.productid\
        ORDER BY meanrating DESC\
        LIMIT 5"

In [7]:
runMyQuery(queryEx1)

,productid,name,meanrating
0,709,"Mountain Bike Socks, M",5.0
1,798,"Road-550-W Yellow, 40",5.0
2,937,HL Mountain Pedal,3.0


### Exercise 2

Much to your disappointment, there are only 3 products with ratings. This is not nearly enough to study the correlation between reviews and total sales. However, your manager wants a description of the highest rated product (the socks) for an upcoming sale. You will have to extract the product description from ```Production.ProductDescription``` by using multiple tables.

(NOTE: It's fine to hard-code the ProductID as this is a one-off study. You will have to start off with the `ProductModelID` in the ```Production.Product``` table. Then go through the ```Production.ProductModelProductDescriptionCulture```. There are multiple product models, but we only want the oldest one (they are going on sale). Use this `ProductDescriptionID` to get the actual description from ```Production.ProductDescription```. Use the documentation provided above if you need help navigating the structure!)

**Description:** Combination of natural and synthetic fibers stays dry and provides just the right cushioning.

**Answer.**

_I have 709 has the productid for the product, so I'll check its productmodelid:_

In [8]:
queryEx2a="SELECT productmodelid\
    FROM Production.Product\
    WHERE productid=709"

In [9]:
runMyQuery(queryEx2a)

,productmodelid
0,18


In [10]:
runMyQuery("SELECT a.productmodelid,productdescriptionid,cultureid,modifieddate\
            FROM Production.ProductModelProductDescriptionCulture as PMPDC\
            INNER JOIN (SELECT productmodelid FROM Production.Product WHERE productid=709) a\
            ON a.productmodelid =PMPDC.productmodelid")

,productmodelid,productdescriptionid,cultureid,modifieddate
0,18,1189,en,2013-04-30
1,18,1459,ar,2013-04-30
2,18,1581,fr,2013-04-30
3,18,1704,th,2013-04-30
4,18,1830,he,2013-04-30
5,18,1956,zh-cht,2013-04-30


_It seems that all descriptions where modified in the same date, so we'll use just the "en"_

In [11]:
runMyQuery("SELECT a.productmodelid,productdescriptionid,cultureid,modifieddate\
            FROM Production.ProductModelProductDescriptionCulture as PMPDC\
            INNER JOIN (SELECT productmodelid FROM Production.Product WHERE productid=709) a\
            ON a.productmodelid =PMPDC.productmodelid\
            WHERE cultureid='en'")

,productmodelid,productdescriptionid,cultureid,modifieddate
0,18,1189,en,2013-04-30


_¿What's in ```ProductDescription```Table?_

In [12]:
runMyQuery("SELECT * FROM Production.ProductDescription ORDER BY productdescriptionid LIMIT 5")

,productdescriptionid,description,rowguid,modifieddate
0,3,Chromoly steel.,301eed3a-1a82-4855-99cb-2afe8290d641,2013-04-30
1,4,Aluminum alloy cups; large diameter spindle.,dfeba528-da11-4650-9d86-cafda7294eb0,2013-04-30
2,5,Aluminum alloy cups and a hollow axle.,f7178da7-1a7e-4997-8470-06737181305e,2013-04-30
3,8,"Suitable for any type of riding, on or off-roa...",8e6746e5-ad97-46e2-bd24-fcea075c3b52,2013-04-30
4,64,This bike delivers a high-level of performance...,7b1c4e90-85e2-4792-b47b-e0c424e2ec94,2013-04-30


_An "aesthetically challenged", yet functional query would give us:_

In [13]:
runMyQuery("SELECT PD.description\
            FROM Production.ProductDescription as PD\
            INNER JOIN(SELECT a.productmodelid,productdescriptionid,cultureid,modifieddate\
                        FROM Production.ProductModelProductDescriptionCulture as PMPDC\
                        INNER JOIN (SELECT productmodelid FROM Production.Product WHERE productid=709) a\
                                    ON a.productmodelid =PMPDC.productmodelid\
                                    WHERE cultureid='en') b\
            ON PD.productdescriptionid=b.productdescriptionid").values.astype(str)

array([['Combination of natural and synthetic fibers stays dry and provides just the right cushioning.']],
      dtype='<U93')

_Which is what we were looking for._

### Exercise 3

Since we cannot infer the most popular product from the reviews, we will calculate the top products by the number of items sold for each product in each product subcategory. Using the ```Sales.SalesOrderDetail``` table, calculate the number of each product sold (don't forget `OrderQty`!). Then you will want to merge with ```Production.Product``` to get the subcategory. The final output should be the products, listed in decreasing order of quantity, with their product and subcategory name. Limit results to 5 rows.

**Answer.**

_I'll start by doing some exploration queries_

In [14]:
runMyQuery("SELECT * FROM Sales.SalesOrderDetail Product LIMIT 3")

,salesorderid,salesorderdetailid,carriertrackingnumber,orderqty,productid,specialofferid,unitprice,unitpricediscount,rowguid,modifieddate
0,43659,1,4911-403C-98,1,776,1,2024.994,0.0,b207c96d-d9e6-402b-8470-2cc176c42283,2011-05-31
1,43659,2,4911-403C-98,3,777,1,2024.994,0.0,7abb600d-1e77-41be-9fe5-b9142cfc08fa,2011-05-31
2,43659,3,4911-403C-98,1,778,1,2024.994,0.0,475cf8c6-49f6-486e-b0ad-afc6a50cdd2f,2011-05-31


In [15]:
runMyQuery("SELECT * FROM Production.Product LIMIT 3")

,productid,name,productnumber,makeflag,finishedgoodsflag,color,safetystocklevel,reorderpoint,standardcost,listprice,size,sizeunitmeasurecode,weightunitmeasurecode,weight,daystomanufacture,productline,class,style,productsubcategoryid,productmodelid,sellstartdate,sellenddate,discontinueddate,rowguid,modifieddate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,0.0,None,None,None,None,0,None,None,None,None,None,2008-04-30,None,None,694215b7-08f7-4c0d-acb1-d734ba44c0c8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,None,0,None,None,None,None,None,2008-04-30,None,None,58ae3c20-4f3a-4749-a7d4-d568806cc537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,None,1,None,None,None,None,None,2008-04-30,None,None,9c21aed2-5bfa-4f18-bcb8-f11638dc2e4e,2014-02-08 10:01:36.827


In [16]:
runMyQuery("SELECT COUNT(*) From Production.Product GROUP BY productsubcategoryid LIMIT 3")

,count
0,3
1,3
2,209


In [17]:
queryEx3a="SELECT productid,sum(orderqty) As total_sales FROM Sales.SalesOrderDetail GROUP BY productid LIMIT 3"

In [18]:
queryEx3a

'SELECT productid,sum(orderqty) As total_sales FROM Sales.SalesOrderDetail GROUP BY productid LIMIT 3'

In [19]:
runMyQuery(queryEx3a)

,productid,total_sales
0,898,15
1,790,812
2,828,94


In [20]:
queryEx3b="SELECT productid,name as product_name,productsubcategoryid FROM Production.Product"

In [21]:
runMyQuery(queryEx3b+" LIMIT 3")

,productid,product_name,productsubcategoryid
0,1,Adjustable Race,None
1,2,Bearing Ball,None
2,3,BB Ball Bearing,None


In [22]:
queryEx3c="SELECT a.productid,b.product_name,b.productsubcategoryid,total_sales \
FROM ("+queryEx3a+") a \
INNER join ("+queryEx3b+") b \
ON a.productid=b.productid \
ORDER BY 4 DESC \
LIMIT 5"

_Including previous queries into the same one, I obtain:_

In [23]:
runMyQuery("SELECT name AS subcategory_name,c.product_name,total_sales_Q \
            FROM Production.ProductSubcategory As PS\
            INNER JOIN (SELECT a.productid,b.product_name,b.productsubcategoryid,total_sales_Q FROM \
            (SELECT productid,sum(orderqty) As total_sales_Q FROM Sales.SalesOrderDetail GROUP BY productid) a \
            INNER JOIN (SELECT productid,name as product_name,productsubcategoryid FROM Production.Product) b \
            ON a.productid=b.productid \
            ORDER BY 4 DESC \
            LIMIT 5) c\
            ON PS.productsubcategoryid=c.productsubcategoryid\
            ORDER BY total_sales_Q DESC")

,subcategory_name,product_name,total_sales_q
0,Caps,AWC Logo Cap,8311
1,Bottles and Cages,Water Bottle - 30 oz.,6815
2,Helmets,"Sport-100 Helmet, Blue",6743
3,Jerseys,"Long-Sleeve Logo Jersey, L",6592
4,Helmets,"Sport-100 Helmet, Black",6532


### Exercise 4

Let's now correlate quantity sold with the price of each item in each subcategory. Alter the previous query to include the price of the item. At the end, group by subcategory and perform a correlation analysis on quantity and price using the ```corr()``` function.

**Answer.**

_We make some arrangements on the same logic, grouping by subcategory to obtain ```corr()```. Is it important to stress that I am going to use **list price** for the calculation, as indicated on Slack's ```#extended-case-questions``` channel._

In [24]:
runMyQuery("SELECT e.categoryname, round(CAST (corr(listprice, total_sales_q) AS numeric), 2) as corrpq \
           FROM (SELECT c.listprice, c.total_sales_Q,d.name AS categoryname \
           FROM (SELECT a.total_sales_q,b.listprice,b.productsubcategoryid FROM(SELECT productid,sum(orderqty) \
           As total_sales_Q FROM Sales.SalesOrderDetail GROUP BY productid) AS a \
            INNER JOIN (SELECT productid,listprice,productsubcategoryid FROM Production.Product) AS b \
            ON a.productid = b.productid) As c \
            INNER JOIN (SELECT productsubcategoryid, name FROM Production.ProductSubcategory) As d \
            ON c.productsubcategoryid=d.productsubcategoryid) As e\
            GROUP BY categoryname")

,categoryname,corrpq
0,Bib-Shorts,0.00
1,Bike Racks,NaN
2,Bike Stands,NaN
3,Bottles and Cages,-0.97
4,Bottom Brackets,1.00
5,Brakes,NaN
6,Caps,NaN
7,Chains,NaN
8,Cleaners,NaN
9,Cranksets,0.49


_I'll leave the ```NaN``` as they are, because they are informative._ 

_____

Notice that some subcategories are degenerate. We expect to see groups like this when they don't have many products. There also seems to be a mixture of correlation coefficients. Looking at this table, we see that socks are negatively correlated with price, which makes sense as they are a substitutable good. However, the touring and mountain bikes have a positive coefficient. This could be due to the fact that people tend to be in those markets to look for a higher-end, quality product.

## Finding our top performing salespeople

As mentioned earlier, we want to see which of our salespeople perform best and whether or not we can incentivize them in an appropriate manner. More specifically, we want to see if the commission percentage we give sales people motivate them to sell more.

### Exercise 5

Query the tables in the ```Sales``` schema to return a table of the salespeople ordered by how much they sold this year (year-to-date). Only show the first 5 rows.

**Answer.**

_At a first glance, it should be enough to Join ```Sales.SalesPerson``` with personal info from ```Person.Person```._

In [25]:
runMyQuery("SELECT firstname,middlename,lastname,cast(salesytd as money) as salesamt FROM \
           (SELECT firstname, middlename, lastname,businessentityid FROM Person.Person) As names \
           INNER JOIN (SELECT salesytd, businessentityid FROM Sales.SalesPerson) As salesytd \
           ON names.businessentityid=salesytd.businessentityid \
           ORDER BY salesamt DESC \
           LIMIT 5")

,firstname,middlename,lastname,salesamt
0,Linda,C,Mitchell,"$4,251,368.55"
1,Jae,B,Pak,"$4,116,871.23"
2,Michael,G,Blythe,"$3,763,178.18"
3,Jillian,None,Carson,"$3,189,418.37"
4,Ranjit,R,Varkey Chudukatil,"$3,121,616.32"


_I'll check consistency on the next exercise._

### Exercise 6

You are skeptical of these sales numbers as it is early on in the year. So you want to go through and validate these by computing the sales for each given salesperson using other tables. Using  ```SalesOrderHeader``` (there is a column `SubTotal`) in the ```Sales``` schema, calculate the sales for each salesperson for **this past year** (2014).  Sales that do not have an associated salesperson is assigned an ID of 0; you should exclude this from the results. Limit results to 5 rows and order by descending total sales.

**Answer.**

_Fist, i'll inspect some rows from the table: sales in 2014_

In [26]:
runMyQuery("SELECT * FROM Sales.SalesOrderHeader LIMIT 1")

,salesorderid,revisionnumber,orderdate,duedate,shipdate,status,onlineorderflag,purchaseordernumber,accountnumber,customerid,salespersonid,territoryid,billtoaddressid,shiptoaddressid,shipmethodid,creditcardid,creditcardapprovalcode,currencyrateid,subtotal,taxamt,freight,totaldue,comment,rowguid,modifieddate
0,43659,8,2011-05-31,2011-06-12,2011-06-07,5,False,PO522145787,10-4020-000676,29825,279,5,985,985,5,16281,105041Vi84182,None,20565.6206,1971.5149,616.0984,23153.2339,None,79b65321-39ca-4115-9cba-8fe0903e12e6,2011-06-07


In [27]:
runMyQuery("SELECT salespersonid,cast(sum(subtotal) as money) salesamt\
           FROM Sales.SalesOrderHeader AS SO\
           WHERE OrderDate>='2014-01-01' AND OrderDate<='2014-12-31' \
           GROUP BY SO.salespersonid \
           ORDER BY salesamt DESC \
           LIMIT 5")

,salespersonid,salesamt
0,NaN,"$8,372,829.73"
1,289.0,"$1,382,996.58"
2,276.0,"$1,271,088.52"
3,275.0,"$1,057,247.38"
4,282.0,"$1,044,810.83"


In [28]:
runMyQuery("SELECT businessentityid,firstname, middlename, lastname FROM Person.Person LIMIT 3")

,businessentityid,firstname,middlename,lastname
0,1,Ken,J,Sánchez
1,2,Terri,Lee,Duffy
2,3,Roberto,None,Tamburello


In [29]:
runMyQuery("SELECT a.businessentityid, a.firstname,a.middlename,a.lastname, b.salesamt FROM \
                    (SELECT businessentityid,firstname, middlename, lastname \
                    FROM Person.Person) As a \
                    INNER JOIN (SELECT DISTINCT salespersonid,cast(sum(subtotal) as money) salesamt \
                               FROM Sales.SalesOrderHeader AS SO\
                               WHERE OrderDate>='2014-01-01' AND OrderDate<='2014-12-31' \
                               GROUP BY SO.salespersonid) As b \
                    ON a.businessentityid=b.salespersonid \
                    ORDER BY salesamt desc \
                    LIMIT 5")

,businessentityid,firstname,middlename,lastname,salesamt
0,289,Jae,B,Pak,"$1,382,996.58"
1,276,Linda,C,Mitchell,"$1,271,088.52"
2,275,Michael,G,Blythe,"$1,057,247.38"
3,282,José,Edvaldo,Saraiva,"$1,044,810.83"
4,277,Jillian,None,Carson,"$1,040,093.41"


### Exercise 7

Looking at the documentation, you see that `SubTotal` in the ```SalesOrderHeader``` table is calculated from other tables in the database. Let us repeat the previous exercise except that we calculate `SubTotal` manually. Using ```SalesOrderDetail``` and ```SalesOrderHeader``` in the ```Sales``` schema, calculate the sales for each salesperson for **this past year** (2014).

**Hint:** You will have to ```JOIN``` ```SalesOrderDetail``` on ```SalesOrderHeader``` to get the salesperson, calculate line totals for each sale using appropriate discounts, then sum all the line totals to get the total sale. Limit results to 5 rows and order by descending total sales. You will want to use ```WITH ... AS``` to save subqueries as we will assume that `SubTotal` doesn't exist moving forward.

**Answer.**

In [30]:
runMyQuery("SELECT * FROM Sales.SalesOrderDetail LIMIT 3")

,salesorderid,salesorderdetailid,carriertrackingnumber,orderqty,productid,specialofferid,unitprice,unitpricediscount,rowguid,modifieddate
0,43659,1,4911-403C-98,1,776,1,2024.994,0.0,b207c96d-d9e6-402b-8470-2cc176c42283,2011-05-31
1,43659,2,4911-403C-98,3,777,1,2024.994,0.0,7abb600d-1e77-41be-9fe5-b9142cfc08fa,2011-05-31
2,43659,3,4911-403C-98,1,778,1,2024.994,0.0,475cf8c6-49f6-486e-b0ad-afc6a50cdd2f,2011-05-31


In [31]:
runMyQuery("SELECT * FROM Sales.SalesOrderHEADER LIMIT 3")

,salesorderid,revisionnumber,orderdate,duedate,shipdate,status,onlineorderflag,purchaseordernumber,accountnumber,customerid,salespersonid,territoryid,billtoaddressid,shiptoaddressid,shipmethodid,creditcardid,creditcardapprovalcode,currencyrateid,subtotal,taxamt,freight,totaldue,comment,rowguid,modifieddate
0,43659,8,2011-05-31,2011-06-12,2011-06-07,5,False,PO522145787,10-4020-000676,29825,279,5,985,985,5,16281,105041Vi84182,NaN,20565.6206,1971.5149,616.0984,23153.2339,None,79b65321-39ca-4115-9cba-8fe0903e12e6,2011-06-07
1,43660,8,2011-05-31,2011-06-12,2011-06-07,5,False,PO18850127500,10-4020-000117,29672,279,5,921,921,5,5618,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738dc42d-d03b-48a1-9822-f95a67ea7389,2011-06-07
2,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,PO18473189620,10-4020-000442,29734,282,6,517,517,5,1346,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,d91b9131-18a4-4a11-bc3a-90b6f53e9d74,2011-06-07


_To obtain salesorderid and amount_

In [32]:
runMyQuery("SELECT DISTINCT salesorderid,cast(SUM(OrderQty*UnitPrice*(1-UnitPriceDiscount)) as money) TotalCalc \
    FROM Sales.SalesOrderDetail \
    GROUP BY salesorderid \
    LIMIT 3")

,salesorderid,totalcalc
0,43659,"$20,565.62"
1,43660,"$1,294.25"
2,43661,"$32,726.48"


_To obtain salesorderid and salespersonid_

In [33]:
runMyQuery("SELECT salesorderid,salespersonid\
    FROM Sales.SalesOrderHeader \
    WHERE orderdate>'2014-01-01' and orderdate<='2014-12-31' \
    LIMIT 3")

,salesorderid,salespersonid
0,63425,None
1,63426,None
2,63427,None


_Joining both tables:_

In [34]:
runMyQuery("SELECT salespersonid,firstname, middlename, lastname,salesamt \
           FROM (SELECT salespersonid, SUM(calculated) as salesamt \
                FROM (SELECT DISTINCT salesorderid,cast(SUM(OrderQty*UnitPrice*(1-UnitPriceDiscount)) as money) \
                        calculated \
                    FROM Sales.SalesOrderDetail \
                    GROUP BY salesorderid) As a \
                    INNER JOIN (SELECT salesorderid,salespersonid\
                                FROM Sales.SalesOrderHeader \
                                WHERE orderdate>'2014-01-01' and orderdate<='2014-12-31') As b \
                    ON a.salesorderid=b.salesorderid \
                    GROUP BY salespersonid \
                ORDER BY salesamt DESC) As c \
           INNER JOIN (SELECT businessentityid,firstname, middlename, lastname FROM Person.Person) As d \
           ON c.salespersonid=d.businessentityid \
           LIMIT 5")

,salespersonid,firstname,middlename,lastname,salesamt
0,289,Jae,B,Pak,"$1,382,996.58"
1,276,Linda,C,Mitchell,"$1,271,088.54"
2,275,Michael,G,Blythe,"$1,057,247.43"
3,282,José,Edvaldo,Saraiva,"$1,044,810.84"
4,277,Jillian,None,Carson,"$1,040,093.41"


_The values are almost identical (some cents difference)_

_______

We see right away that there are descrepencies between the two sales totals. For the remainder of the exercise you will use your answer from exercise 2 as you are told the SalesYTD in the SalesPerson table is for the company's fiscal year and not the calendar year.

### Exercise 8

The next thing we are curious about is whether or not commision percentage impacts total sales. Using ```corr()```, see if there is a positive relationship between total sales and commission percentage. (If you regress on commission percentage to total sales, the slope will be massive, so normalize by the average of the total sales.)

**Answer.**

_Since the problem is not specifying a particular group of Sales Persons, I'll correlate total sales and commission percentage for all of them._ 

In [35]:
runMyQuery("SELECT a.salespersonid,a.salesamt,b.commissionpct \
        FROM(SELECT salespersonid, sum(subtotal) as salesamt \
            FROM Sales.SalesOrderHeader \
            GROUP BY salespersonid) As a \
        INNER JOIN (SELECT businessentityid,commissionpct FROM Sales.SalesPerson) As b \
        ON a.salespersonid=b.businessentityid")

,salespersonid,salesamt,commissionpct
0,274,1.092124e+06,0.000
1,275,9.293903e+06,0.012
2,276,1.036701e+07,0.015
3,277,1.006580e+07,0.015
4,278,3.609447e+06,0.010
5,279,7.171013e+06,0.010
6,280,3.325103e+06,0.010
7,281,6.427006e+06,0.010
8,282,5.926418e+06,0.015
9,283,3.729945e+06,0.012


_And correlating the two columns:_

In [36]:
runMyQuery("SELECT round(CAST(corr(c.salesamt, c.commissionpct) AS numeric),2) AS corrcomsal\
            FROM (SELECT a.salespersonid,a.salesamt,b.commissionpct \
                FROM(SELECT salespersonid, sum(subtotal) AS salesamt \
                FROM Sales.SalesOrderHeader \
                GROUP BY salespersonid) AS a \
                INNER JOIN (SELECT businessentityid,commissionpct FROM Sales.SalesPerson) AS b \
                ON a.salespersonid=b.businessentityid) As c")

,corrcomsal
0,0.44


### Exercise 9

You now just realized that your company is multinational and has offices in different countries around the world. Furthermore, you are told the sales are recorded in the local currency so you must convert it to USD if you wish to compare the different salespeople. Construct a new ```Sales.SalesPerson``` table to include a new column for ```CurrencyCode```. You will have to do this by extracting the `TerritoryID` of the salesperson from ```Sales.SalesTerritory``` then use `CountryRegionCode` in that table to get `CurrencyCode` from ```Sales.CountryRegionCurrency```. This will involve multiple ```JOIN```'s. Limit results to 5 rows.

**Answer.**

_Inspecting original table:_

In [37]:
runMyQuery("SELECT * \
            FROM Sales.SalesPerson AS SP ")

,businessentityid,territoryid,salesquota,bonus,commissionpct,salesytd,saleslastyear,rowguid,modifieddate
0,274,NaN,NaN,0.0,0.000,5.596976e+05,0.000000e+00,48754992-9ee0-4c0e-8c94-9451604e3e02,2010-12-28
1,275,2.0,300000.0,4100.0,0.012,3.763178e+06,1.750406e+06,1e0a7274-3064-4f58-88ee-4c6586c87169,2011-05-24
2,276,4.0,250000.0,2000.0,0.015,4.251369e+06,1.439156e+06,4dd9eee4-8e81-4f8c-af97-683394c1f7c0,2011-05-24
3,277,3.0,250000.0,2500.0,0.015,3.189418e+06,1.997186e+06,39012928-bfec-4242-874d-423162c3f567,2011-05-24
4,278,6.0,250000.0,500.0,0.010,1.453719e+06,1.620277e+06,7a0ae1ab-b283-40f9-91d1-167abf06d720,2011-05-24
5,279,5.0,300000.0,6700.0,0.010,2.315186e+06,1.849641e+06,52a5179d-3239-4157-ae29-17e868296dc0,2011-05-24
6,280,1.0,250000.0,5000.0,0.010,1.352577e+06,1.927059e+06,be941a4a-fb50-4947-bda4-bb8972365b08,2011-05-24
7,281,4.0,250000.0,3550.0,0.010,2.458536e+06,2.073506e+06,35326ddb-7278-4fef-b3ba-ea137b69094e,2011-05-24
8,282,6.0,250000.0,5000.0,0.015,2.604541e+06,2.038235e+06,31fd7fc1-dc84-4f05-b9a0-762519eacacc,2011-05-24
9,283,1.0,250000.0,3500.0,0.012,1.573013e+06,1.371635e+06,6bac15b2-8ffb-45a9-b6d5-040e16c2073f,2011-05-24


_Each ```territoryid``` belongs to a Country, which has a corresponding ```currencycode```._

In [38]:
runMyQuery("WITH tblCurrencies AS (\
           SELECT a.territoryid,a.countryregioncode,b.currencycode \
            FROM(SELECT DISTINCT territoryid,countryregioncode FROM Sales.SalesTerritory) As a \
            INNER JOIN (SELECT DISTINCT countryregioncode,CurrencyCode FROM Sales.CountryRegionCurrency) As b \
            ON a.countryregioncode=b.countryregioncode) \
            SELECT * FROM tblCurrencies")

,territoryid,countryregioncode,currencycode
0,10,GB,GBP
1,3,US,USD
2,2,US,USD
3,4,US,USD
4,5,US,USD
5,1,US,USD
6,6,CA,CAD
7,8,DE,EUR
8,8,DE,DEM
9,9,AU,AUD


_Reordering and merging these queries, we can add a ```Currencycode```field by. I'll be using Top 5 in ```salesytd```:_

In [39]:
runMyQuery("WITH tblCurrencies AS (\
            SELECT a.territoryid, b.currencycode \
            FROM(SELECT DISTINCT territoryid,countryregioncode FROM Sales.SalesTerritory) As a \
            INNER JOIN (SELECT DISTINCT countryregioncode,CurrencyCode FROM Sales.CountryRegionCurrency) As b \
            ON a.countryregioncode=b.countryregioncode) \
            SELECT businessentityid,SP.territoryid,salesquota,bonus,commissionpct,salesytd,saleslastyear,rowguid, \
                modifieddate,currencycode \
            FROM Sales.SalesPerson AS SP \
            JOIN tblCurrencies \
            ON tblCurrencies.territoryid=SP.territoryid \
            WHERE currencycode<>'FRF' AND currencycode<>'DEM' \
            ORDER BY salesytd DESC \
            LIMIT 5")

,businessentityid,territoryid,salesquota,bonus,commissionpct,salesytd,saleslastyear,rowguid,modifieddate,currencycode
0,276,4,250000.0,2000.0,0.015,4.251369e+06,1.439156e+06,4dd9eee4-8e81-4f8c-af97-683394c1f7c0,2011-05-24,USD
1,289,10,250000.0,5150.0,0.020,4.116871e+06,1.635823e+06,25f6838d-9db4-4833-9ddc-7a24283af1ba,2012-05-23,GBP
2,275,2,300000.0,4100.0,0.012,3.763178e+06,1.750406e+06,1e0a7274-3064-4f58-88ee-4c6586c87169,2011-05-24,USD
3,277,3,250000.0,2500.0,0.015,3.189418e+06,1.997186e+06,39012928-bfec-4242-874d-423162c3f567,2011-05-24,USD
4,290,7,250000.0,985.0,0.016,3.121616e+06,2.396540e+06,f509e3d4-76c8-42aa-b353-90b7b8db08de,2012-05-23,EUR


_A row is dropped, where ```territoryid``` was unspecified. Also, we dropped "vintage" (as of 2014) currencies._

### Exercise 10

Now that we have the currency codes associated with each salesperson, redo Exercise 7 to take the currency exchange into account.

**Hint:** the rates in the```Sales.CurrencyRate``` table always go from `FromCurrencyCode='USD'` to `ToCurrencyCode='Desired Currency Code'` and they are listed every day. When calculating line totals, use the `AverageRate` for that day. You should be able to reuse a lot of Exercise 3.

**Answer.**

_Original sales table for 2014 was:_

In [40]:
runMyQuery("SELECT salespersonid,firstname, middlename, lastname,salesamt \
           FROM (SELECT salespersonid, SUM(calculated) as salesamt \
                FROM (SELECT DISTINCT salesorderid,SUM(OrderQty*UnitPrice*(1-UnitPriceDiscount))as calculated \
                    FROM Sales.SalesOrderDetail \
                    GROUP BY salesorderid) As a \
                    INNER JOIN (SELECT salesorderid,salespersonid\
                                FROM Sales.SalesOrderHeader \
                                WHERE orderdate>'2014-01-01' and orderdate<='2014-12-31') As b \
                    ON a.salesorderid=b.salesorderid \
                    GROUP BY salespersonid \
                ORDER BY salesamt DESC) As c \
           INNER JOIN (SELECT businessentityid,firstname, middlename, lastname FROM Person.Person) As d \
           ON c.salespersonid=d.businessentityid \
           ORDER BY salesamt DESC")

,salespersonid,firstname,middlename,lastname,salesamt
0,289,Jae,B,Pak,1.382997e+06
1,276,Linda,C,Mitchell,1.271089e+06
2,275,Michael,G,Blythe,1.057247e+06
3,282,José,Edvaldo,Saraiva,1.044811e+06
4,277,Jillian,None,Carson,1.040093e+06
5,290,Ranjit,R,Varkey Chudukatil,8.675186e+05
6,279,Tsvi,Michael,Reiter,7.872044e+05
7,281,Shu,K,Ito,7.779417e+05
8,284,Tete,A,Mensa-Annan,6.009972e+05
9,286,Lynn,N,Tsoflias,5.857558e+05


_Now I obtain currencies for each ```businessentityid```_

In [41]:
runMyQuery("WITH tblCurrencies AS (\
            SELECT a.territoryid, b.currencycode \
            FROM(SELECT DISTINCT territoryid,countryregioncode FROM Sales.SalesTerritory) As a \
            INNER JOIN (SELECT DISTINCT countryregioncode,CurrencyCode FROM Sales.CountryRegionCurrency) As b \
            ON a.countryregioncode=b.countryregioncode) \
            SELECT businessentityid,currencycode \
            FROM Sales.SalesPerson AS SP \
            JOIN tblCurrencies \
            ON tblCurrencies.territoryid=SP.territoryid \
            WHERE currencycode<>'FRF' AND currencycode <>'DEM'")

,businessentityid,currencycode
0,275,USD
1,276,USD
2,277,USD
3,278,CAD
4,279,USD
5,280,USD
6,281,USD
7,282,CAD
8,283,USD
9,284,USD


_And the average exchange rate for 2014:_

In [42]:
runMyQuery("SELECT tocurrencycode, 1/avg(averagerate) as factor \
            FROM Sales.CurrencyRate \
            WHERE currencyratedate>='2014-01-01' AND currencyratedate<='2014-12-31'\
            GROUP BY tocurrencycode")

,tocurrencycode,factor
0,GBP,1.549680
1,ARS,0.275057
2,MXN,0.100215
3,CAD,0.639946
4,BRL,0.302859
5,EUR,0.983822
6,CNY,0.120674
7,VEB,0.000720
8,USD,1.000000
9,SAR,0.266629


In [43]:
runMyQuery("SELECT z.salespersonid,z.firstname, z.middlename, z.lastname, \
            cast((z.salesamt*y.factor) as money) as Sales_USD \
           FROM(SELECT salespersonid,firstname, middlename, lastname,salesamt \
           FROM (SELECT salespersonid, SUM(calculated) as salesamt \
                FROM (SELECT DISTINCT salesorderid,SUM(OrderQty*UnitPrice*(1-UnitPriceDiscount))as calculated \
                    FROM Sales.SalesOrderDetail \
                    GROUP BY salesorderid) As a \
                    INNER JOIN (SELECT salesorderid,salespersonid\
                                FROM Sales.SalesOrderHeader \
                                WHERE orderdate>'2014-01-01' and orderdate<='2014-12-31') As b \
                    ON a.salesorderid=b.salesorderid \
                    GROUP BY salespersonid \
                ORDER BY salesamt DESC) As c \
           INNER JOIN (SELECT businessentityid,firstname, middlename, lastname FROM Person.Person) As d \
           ON c.salespersonid=d.businessentityid) As z \
           INNER JOIN \
           (SELECT a.businessentityid,b.factor \
            FROM (WITH tblCurrencies AS (\
            SELECT a.territoryid, b.currencycode \
            FROM(SELECT DISTINCT territoryid,countryregioncode FROM Sales.SalesTerritory) As a \
            INNER JOIN (SELECT DISTINCT countryregioncode,CurrencyCode FROM Sales.CountryRegionCurrency) As b \
            ON a.countryregioncode=b.countryregioncode) \
            SELECT businessentityid,currencycode \
            FROM Sales.SalesPerson AS SP \
            JOIN tblCurrencies \
            ON tblCurrencies.territoryid=SP.territoryid \
            WHERE currencycode<>'FRF' AND currencycode <>'DEM'"") AS a \
            INNER JOIN (SELECT tocurrencycode, 1/avg(averagerate) as factor \
            FROM Sales.CurrencyRate \
            WHERE currencyratedate>='2014-01-01' AND currencyratedate<='2014-12-31'\
            GROUP BY tocurrencycode) AS b \
            ON a.currencycode=b.tocurrencycode)As y \
           ON z.salespersonid=y.businessentityid \
           ORDER BY sales_usd DESC \
           LIMIT 5")

,salespersonid,firstname,middlename,lastname,sales_usd
0,289,Jae,B,Pak,"$2,143,201.52"
1,276,Linda,C,Mitchell,"$1,271,088.52"
2,275,Michael,G,Blythe,"$1,057,247.38"
3,277,Jillian,None,Carson,"$1,040,093.41"
4,290,Ranjit,R,Varkey Chudukatil,"$853,484.17"


_And now we have corrected all values to USD, using 2014 average exchange rate._

### Exercise 11

How does the correlation change from Exercise 4 once you've adjusted for the currency?

**Answer.**

_It wouldn't change, since we used the current ```ListPrice```  from the product table for the exercise, and did the correlation coefficient with sold quantity by each subcategory. Had we used the actual sold price, I should have to adjust it via exchange rate for that date._